In [1]:

from ROOT import TMVA, TFile, TTree, TCut
from subprocess import call
from os.path import isfile
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
 

Welcome to JupyROOT 6.24/06


2022-02-10 07:42:54.291691: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.

In [2]:

# Setup TMVA
TMVA.Tools.Instance()
TMVA.PyMethodBase.PyInitialize()

# Fastest (and smallest) dataset
#data = TFile.Open('10k_sample_data.root')
# Medium size dataset
data = TFile.Open('50k_sample_data.root')
signal = data.Get('Mixing')
background = data.Get('Cosmic')




In [3]:

dataloader = TMVA.DataLoader('dataset')

variable_list = [
   'nhits',
   'residual',
   'r',
   'S0rawPerp',
   'S0axisrawZ',
   'phi_S0axisraw',
   'nCT',
   'nGT',
   'tracksdca',
   'curvemin',
   'curvemean',
   'lambdamin',
   'lambdamean',
   'curvesign',
   # Want to see the final tests fail? Enable these
   #'phi',
   #'X',
   #'Y',
   #'Z',
]
# Assign the variables for training (and testing)
for variable in variable_list:
    dataloader.AddVariable(variable)

In [4]:

dataloader.AddSignalTree(signal, 1.0)
dataloader.AddBackgroundTree(background, 1.0)

testing_events = 4000

dataloader.PrepareTrainingAndTestTree(TCut(''),
                                      'nTrain_Signal=' +
                                      str(signal.GetEntries() - testing_events) + 
                                      ':nTrain_Background=' +
                                      str(background.GetEntries() - testing_events) + 
                                      ':SplitMode=Random:NormMode=NumEvents:!V')
 

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree Mixing of type Signal with 50000 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree Cosmic of type Background with 50000 events
                         : Dataset[dataset] : Class index : 0  name : Signal
                         : Dataset[dataset] : Class index : 1  name : Background


# Select the classifiers of interest here:

In [5]:
# Maybe pick one or two out of this list if memory is limited
active_mva_list = [
    'Fisher',
    'PyKeras',
    'Cuts',
    'KNN',
    #'SVM',
    'BDT',
    'BDTB',
    'BDTG',
]
 

In [6]:
output = TFile.Open('TMVA.root', 'RECREATE')
factory = TMVA.Factory('TMVAClassification', output,
                       '!V:!Silent:Color:DrawProgressBar:AnalysisType=Classification')


# Book methods

if 'Fisher' in active_mva_list:
    factory.BookMethod(dataloader, TMVA.Types.kFisher, 'Fisher',
                       '!H:!V:Fisher')
if 'PyKeras' in active_mva_list:
    # Generate model
    
    # Define model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=len(variable_list)))
    model.add(Dense(64, activation='relu', input_dim=len(variable_list)))
    model.add(Dense(2, activation='softmax'))
    
    # Set loss and optimizer
    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=0.01), metrics=['accuracy', ])
    # Store model to file
    model.save('model.h5')
    model.summary()
    factory.BookMethod(dataloader, TMVA.Types.kPyKeras, 'PyKeras',
                       'H:!V:FilenameModel=model.h5:NumEpochs=20:BatchSize=32')
# Cut optimisation
if 'Cuts' in active_mva_list:
    factory.BookMethod( dataloader, TMVA.Types.kCuts, 'Cuts',
                        '!H:!V:FitMethod=MC:EffSel:SampleSize=200000:VarProp=FSmart' )
#// K-Nearest Neighbour classifier (KNN)
if 'KNN' in active_mva_list:
    factory.BookMethod( dataloader, TMVA.Types.kKNN, 'KNN',
                       '!H:nkNN=20:ScaleFrac=0.8:SigmaFact=1.0:Kernel=Gaus:UseKernel=F:UseWeight=T:!Trim' )

# Support Vector Machine
if 'SVM' in active_mva_list:
    factory.BookMethod( dataloader, TMVA.Types.kSVM, 'SVM', 'Gamma=0.25:Tol=0.001:VarTransform=Norm' )

# "BDT"  // Adaptive Boost
if 'BDT' in active_mva_list:
    factory.BookMethod( dataloader, TMVA.Types.kBDT, 'BDT',
                       '!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20:SigToBkgFraction=1.0' )

# "BDTB" // Bagging random forest
if 'BDTB' in active_mva_list:
    factory.BookMethod( dataloader, TMVA.Types.kBDT, 'BDTB',
                      '!H:!V:NTrees=500:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=1.0:SeparationType=GiniIndex:nCuts=50:MaxDepth=4:MinNodeSize=5.0%:UseRandomisedTrees:UseNvars=4:CreateMVAPdfs' )

# "BDTG" // Gradient Boost
if 'BDTG' in active_mva_list:  
    factory.BookMethod( dataloader, TMVA.Types.kBDT, 'BDTG',
                        '!H:!V:NTrees=1000:MinNodeSize=2.5%:BoostType=Grad:Shrinkage=0.10:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=2:SigToBkgFraction=1.0' )
 

/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________
Factory                  : Booking method: Fisher
                         : 
Factory                  : Booking method: PyKeras
                         : 
                         : Setting up keras with 
                         : 
                         : 
<WARNING>                : 
                         : 
                         : 
                         : 
Factory  

2022-02-10 07:43:03.624874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cvmfs/sft.cern.ch/lcg/releases/MCGenerators/thepeg/2.2.1-8d929/x86_64-centos7-gcc8-opt/lib/ThePEG:/cvmfs/sft.cern.ch/lcg/releases/MCGenerators/herwig++/7.2.1-f3599/x86_64-centos7-gcc8-opt/lib/Herwig:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/torch/lib:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/contrib/tensor_forest:/cvmfs/sft.cern.ch/lcg/views/LCG_101swan/x86_64-centos7-gcc8-opt/lib/python3.9/site-packages/tensorflow/python/framework:/cvmfs/sft.cern.ch/lcg/releases/java/8u222-884d8/x86_64-centos7-gcc8-opt/jre/lib/amd64:/cvmfs/sft.cern.ch/lc

# Train our classifiers! This can take some time

In [7]:

# Run training, test and evaluation
factory.TrainAllMethods()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 5,250
Trainable params: 5,250
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
2300/2300 [==============================] - 8s 3ms/step - loss: 15199135270445252608.0000 - accuracy: 0.4990 - val_loss: 0.6929 - val_accuracy: 0.5041

Epoch 00001: val_loss improved from inf to 0.69293, saving model to dataset/weights/TrainedModel_PyKeras.h5
Epoch 2/20
2300/2300 [==============================] - 4s 2ms/step - loss: 0.6934 - accuracy: 0.4990 - val_loss

0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
2022-02-10 07:43:22.343452: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-10 07:43:22.346620: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2399930000 Hz
0%, time left: unknown
7%, time left: 5 sec
13%, time left: 4 sec
19%, time left: 4 sec
25%, time left: 3 sec
32%, time left: 3 sec
38%, time left: 2 sec
44%, time left: 2 sec
50%, time left: 2 sec
57%, time left: 2 sec
63%, time left: 1 sec
69%, time left: 1 sec
75%, time left: 1 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7

In [8]:
factory.TestAllMethods()

Factory                  : Test all methods
Factory                  : Test method: Fisher for Classification performance
                         : 
Fisher                   : [dataset] : Evaluation of Fisher on testing sample (8000 events)
                         : Elapsed time for evaluation of 8000 events: 0.00444 sec       
Factory                  : Test method: PyKeras for Classification performance
                         : 
                         : Setting up tf.keras
                         : Using TensorFlow version 2
                         : Use Keras version from TensorFlow : tf.keras
                         :  Loading Keras Model 
                         : Loaded model from file: dataset/weights/TrainedModel_PyKeras.h5
PyKeras                  : [dataset] : Evaluation of PyKeras on testing sample (8000 events)
                         : Elapsed time for evaluation of 8000 events: 0.358 sec       
Factory                  : Test method: Cuts for Classification per

0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7%, time left: 0 sec
13%, time left: 0 sec
19%, time left: 0 sec
25%, time left: 0 sec
32%, time left: 0 sec
38%, time left: 0 sec
44%, time left: 0 sec
50%, time left: 0 sec
57%, time left: 0 sec
63%, time left: 0 sec
69%, time left: 0 sec
75%, time left: 0 sec
82%, time left: 0 sec
88%, time left: 0 sec
94%, time left: 0 sec
0%, time left: unknown
7%, time left: 11 sec
13%, time left: 9 sec
19%, time left: 8 sec
25%, time left: 7 sec
32%, time left: 7 sec
38%, time left: 6 sec
44%, time left: 5 sec
50%, time left: 4 sec
57%, time left: 4 sec
63%, time left: 3 sec
69%, time left: 3 sec
75%, time left: 2 sec
82%, time

In [9]:
factory.EvaluateAllMethods()

Factory                  : Evaluate all methods
Factory                  : Evaluate classifier: Fisher
                         : 
Fisher                   : [dataset] : Loop over test events and fill histograms with classifier response...
                         : 
TFHandler_Fisher         :      Variable             Mean             RMS     [        Min             Max ]
                         : ------------------------------------------------------------------------------------
                         :         nhits:         57.458         1707.6   [         3.0000         91517. ]
                         :      residual:         2.2079         4.8267   [        -1.0000         45.366 ]
                         :             r:         3.2913         3.3755   [         0.0000         46.386 ]
                         :     S0rawPerp:        0.88420        0.13544   [        0.48613         1.0000 ]
                         :    S0axisrawZ:        -6.6796         24.842   [    

In [10]:
# The TMVA.root file has lots of nice diagnostic plots inside it now:
output.ls()

# TMVA::Gui() in the root program doesn't have a batch mode yet (so I don't believe it can be used from jupyter)

TFile**		TMVA.root	
 TFile*		TMVA.root	
  TDirectoryFile*		dataset	dataset
   TDirectoryFile*		InputVariables_Id	InputVariables_Id
    TDirectoryFile*		CorrelationPlots	CorrelationPlots
     KEY: TH2F	scat_residual_vs_nhits_Signal_Id;1	residual versus nhits (Signal)_Id
     KEY: TProfile	prof_residual_vs_nhits_Signal_Id;1	profile residual versus nhits (Signal)_Id
     KEY: TH2F	scat_residual_vs_nhits_Background_Id;1	residual versus nhits (Background)_Id
     KEY: TProfile	prof_residual_vs_nhits_Background_Id;1	profile residual versus nhits (Background)_Id
     KEY: TH2F	scat_r_vs_nhits_Signal_Id;1	r versus nhits (Signal)_Id
     KEY: TProfile	prof_r_vs_nhits_Signal_Id;1	profile r versus nhits (Signal)_Id
     KEY: TH2F	scat_r_vs_nhits_Background_Id;1	r versus nhits (Background)_Id
     KEY: TProfile	prof_r_vs_nhits_Background_Id;1	profile r versus nhits (Background)_Id
     KEY: TH2F	scat_S0rawPerp_vs_nhits_Signal_Id;1	S0rawPerp versus nhits (Signal)_Id
     KEY: TProfile	prof_S0rawPer

In [11]:
output.Close()


# Test on a real experiment (not labeled training data)

In [17]:
from array import array
from ROOT import TString
import numpy as np
experiment_data = TFile.Open('sample_experiment.root')
testsignal = experiment_data.Get('sample_experiment')


reader = TMVA.Reader( "!Color:!Silent" );
#for variable in variable_list:
#    reader.AddVariable(variable)

    
    
branches = {}
for branchName in variable_list:
    branches[branchName] = array('f', [-999])
    reader.AddVariable(branchName, branches[branchName])
    testsignal.SetBranchAddress(branchName, branches[branchName])
 
# Book methods
#reader.BookMVA('PyKeras', TString('dataset/weights/TMVAClassification_PyKeras.weights.xml'))
for method in active_mva_list:
    reader.BookMVA(TString(method), TString('dataset/weights/TMVAClassification_' + method + '.weights.xml'))

# Print some example classifications
print('Some signal example classifications:')


'''
From TMVA Gui:

--- ==================================================================================================
--- Classifier   (  #signal, #backgr.)  Optimal-cut  S/sqrt(S+B)      NSig      NBkg   EffSig   EffBkg
--- --------------------------------------------------------------------------------------------------
---     Fisher:  (       20,     2000)       0.7180      2.22393     12.96        21    0.648   0.0105
---    PyKeras:  (       20,     2000)       0.4958     0.445049        20    1999.5        1   0.9998
---       Cuts:  (       20,     2000)       0.0250     0.187192     0.415       4.5  0.02075  0.00225
---        KNN:  (       20,     2000)       0.9501      2.93631    13.435       7.5   0.6718  0.00375
---        BDT:  (       20,     2000)       0.1774       3.2772    14.455         5   0.7228   0.0025
---       BDTB:  (       20,     2000)       0.1678      3.29334    13.955         4   0.6977    0.002
---       BDTG:  (       20,     2000)       0.9736      3.34119    12.895         2   0.6448    0.001
--- --------------------------------------------------------------------------------------------------

'''

threshold_list = [
     0.7180,
     0.4958,
     0.0250,
     0.9501,
     0.1774,
     0.1678,
     0.9736,
]
efficiency_list = [
    0.648,
    1,
    0.02075,
    0.6718,
    0.7228,
    0.6977,
    0.6448,
]


for method, threshold, efficiency in zip(active_mva_list, threshold_list, efficiency_list):
    print(method)
    counts = 0
    mean_model_output = 0.0
    for i in range(testsignal.GetEntries()):
        testsignal.GetEntry(i)
        mean_model_output += reader.EvaluateMVA(TString(method))
        if reader.EvaluateMVA(TString(method)) > threshold:
            counts += 1
        #print(reader.EvaluateMVA(TString(method)))
    print('Mean model output ' + str ( mean_model_output / testsignal.GetEntries() ))
    print( '\tTotal Signal = ' + str(counts) + '/' + str(testsignal.GetEntries()))
    print( '\tEstimated Population = ' + str(counts/efficiency) + ' +/- ' + str( np.sqrt(counts)/ efficiency) + '(' + str(testsignal.GetEntries() - 2.1*10) + ')')
print('')

Some signal example classifications:
Fisher
Mean model output 0.7637989497413414
	Total Signal = 325/539
	Estimated Population = 501.5432098765432+/-27.820611693394977(518.0)
PyKeras
Mean model output 0.49629825353622437
	Total Signal = 539/539
	Estimated Population = 539.0+/-23.2163735324878(518.0)
Cuts
Mean model output 0.0
	Total Signal = 0/539
	Estimated Population = 0.0+/-0.0(518.0)
KNN
Mean model output 0.8575139146567715
	Total Signal = 346/539
	Estimated Population = 515.034236379875+/-27.68841208356397(518.0)
BDT
Mean model output 0.22220783780527611
	Total Signal = 365/539
	Estimated Population = 504.9806308799115+/-26.431894264724402(518.0)
BDTB
Mean model output 0.1980276833405108
	Total Signal = 354/539
	Estimated Population = 507.3813960154794+/-26.967016944570414(518.0)
BDTG
Mean model output 0.728210852174187
	Total Signal = 332/539
	Estimated Population = 514.8883374689826+/-28.258168669802416(518.0)

                         : Booking "Fisher" of type "Fisher" from da